In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import os
os.chdir('/content/drive/My Drive/Projects/reddit-vote-predictor')
import multiprocessing as mp
import sys
import shutil as sh
import pandas as pd
import numpy as np
import statsmodels.api as sm
import datetime as dt
import matplotlib.pyplot as plt
from scipy import stats as s
from skimage import io
from mpl_toolkits.axes_grid1 import ImageGrid
import datetime as dt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision import models
# For object detection model:
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from PIL import Image, ImageDraw
# For loading images from URL:
import requests
from io import BytesIO
from tqdm import tqdm

In [ ]:
print(torch.cuda.is_available())  # Should print True
print(torch.cuda.get_device_name(0))  # Prints the GPU name

True
Tesla T4


#### Read in data

The previous scripts did the following
1. Scrape posts from a given subreddit (in this case r/dog and r/cat) within a given time frame
2. Delete posts not containing an image as the `post_hint`
3. Then use Reddit's API to pull additional data, such as `upvote_ratio`
4. Delete posts that have one upvote since this is indistinguishable from posts that received the default one upvote upon posting and nothing else.


In [ ]:
dog = pd.read_csv('data/tbl/dog_praw.csv', index_col = None)
dog = dog.sort_values(by = 'created_date').reset_index(drop = True)
cat = pd.read_csv('data/tbl/cat_praw.csv', index_col = None)
cat = cat.sort_values(by = 'created_date').reset_index(drop = True)

#### Further filtering

1. Check that the image is hosted on reddit, else setup the script to download the thumbnail.
2. Then, check that the resultant link brings us to an image.

In [ ]:
# If the image is not hosted on reddit, then use the url to the thumbnail
dog['FEATURE'] = dog.url
condition = ~dog.url.str.contains('redd.it')
dog.loc[condition, 'FEATURE'] = dog.loc[condition, 'thumbnail']
# for cat
cat['FEATURE'] = cat.url
condition = ~cat.url.str.contains('redd.it')
cat.loc[condition, 'FEATURE'] = cat.loc[condition, 'thumbnail']

In [ ]:
# Check that images exist within the above-filtered url's
def check_image_exists(url):
    try:
        response = requests.get(url)
        return response.status_code == 200
    except requests.RequestException:
        return False

# Add a new column 'image_found' indicating if an image exists at the URL
#dog['image_found'] = dog['FEATURE'].apply(check_image_exists)
#cat['image_found'] = cat['FEATURE'].apply(check_image_exists)
# Also, parallelize this because it can take a while
n_cores = mp.cpu_count()
pool = mp.Pool(processes = n_cores)
dog['image_found'] = pool.map(check_image_exists, dog['FEATURE'])
cat['image_found'] = pool.map(check_image_exists, cat['FEATURE'])
pool.close()
pool.join()

In [ ]:
print(f"{round(dog[dog['image_found']].shape[0] / dog.shape[0], 2)} percent of dog images exist.")
print(f"{round(cat[cat['image_found']].shape[0] / cat.shape[0], 2)} percent of cat images exist")

0.84 percent of dog images exist.
0.86 percent of cat images exist


#### Save the data frame with the new variable indicating the images that exist

In [ ]:
dog[dog['image_found']].to_csv('data/tbl/dog_good-url.csv', index = False)
cat[cat['image_found']].to_csv('data/tbl/cat_good-url.csv', index = False)

#### Quick analysis of targets

1. Check temporal trend in `upvote_ratio` and `upvote`

In [ ]:
# required modules (already loaded as above)
#import numpy as np
#import pandas as pd
#import matplotlib.pyplot as plt

def detrend_data(df, col_name, window_size=100, new_col_name = 'detrended'):
    """
    Detrend the data of a given column, creating a new column.

    Args:
        df (pd.DataFrame): DataFrame with 'Datetime' and 'Value' columns.
        col_name (str): Column name with continuous data to de-trend.
        window_size (int): Size of the moving average window.
        new_col_name (str): New column name with de-trended data.

    Returns:
        pd.DataFrame: DataFrame with an additional 'Detrended' column.
    """
    df[new_col_name] = df[col_name].rolling(window=window_size, min_periods=1).mean()
    return df

def make_plots(df, original, detrended, date_col):
    """
    Plot both the original data and the detrended data.

    Args:
        df (pd.DataFrame): DataFrame with `date_col`, `original`, and `detrended` columns.
        original (str): column name of original data
        detrended (str): column name of detrended column `original`
        date_col (str): x-axis, column of timestamps/dates
    """
    plt.figure(figsize=(12, 6))

    # Plot the original data without a trend
    plt.subplot(2, 1, 1)
    plt.plot(df.index, df[original], marker='o', linestyle='-', label='Data without Trend', color='b')
    plt.xlabel(date_col)
    plt.ylabel(original)
    plt.title(f'Original {original} without a Linear Trend')
    plt.grid(True)
    plt.legend()

    # Plot the detrended data using the moving average filter
    plt.subplot(2, 1, 2)
    plt.plot(df.index, df[detrended], marker='o', linestyle='-', label='Detrended Data (Moving Average)', color='g')
    plt.xlabel(date_col)
    plt.ylabel(detrended)
    plt.title(f'Detrended {original} using Moving Average (Daily Granularity)')
    plt.grid(True)
    plt.legend()

    plt.tight_layout()
    plt.show()

# Create a sample DataFrame with datetime and value columns
def plot_detrended_data(df_, original_, dat_col_):
    new_data = detrend_data(df_, original_, window_size=50, new_col_name = 'detrended')
    make_plots(new_data, original_, 'detrended', 'created_date')

In [ ]:
# Plot dog upvote ratio
plot_detrended_data(dog, 'upvote_ratio', 'created_date')
# Plot cat upvote ratio
plot_detrended_data(cat, 'upvote_ratio', 'created_date')
# Plot dog upvotes
plot_detrended_data(dog, 'upvotes', 'created_date')
# Plot cat upvotes
plot_detrended_data(cat, 'upvotes', 'created_date')

#### Conclusions:

Upvote ratio doesn't have much of a trend, so just use as is.

For upvotes, larger window size would be better.

After analyzing the targets, upvote ratio is the better feature to predict since it is robust to autocorrelation unlike upvotes.

## Preprocessing

1. Use a dog (or cat) detector to bound a box around a dog/cat
2. If none is found, don't use the image.
3. Crop around the bounding box using a center crop, but the center is the center of the bb
4. Save the images in a folder so that the dataloader doesn't have to deal with
skipping images if the above yields no image.
4. Transforms the dataloader will take care of:

  *   Resize all images
  *   Check color format is consistent across images
  *   Convert images to tensor

In [1]:
# Make a helper function to filter for images that have a good url to an image
def fd(df_):
  """
  -F-ilter the -D-ata using `image_found`
  """
  return df_[df_['image_found']]

### Object Detection

1. Determine if there is a dog (or cat) detected
2. If multiple, find the largest one using the area of the bounding boxes
3. Obtain the center of the largest bounding box corresponding to the dog (or cat) for each image in which one is found

#### Parameterize object detection model



*   Download `faster r-cnn resnet 50` with pretrained weights
*   Use the model to predict on our images of dogs or cats
* The label for dog will be 18 and 17 for cat
* Add a transform that converts images to tensors
* Add a step to ensure the images are RGB color format



In [ ]:
# Download the model and define the transform in advance rather than each time.
model = fasterrcnn_resnet50_fpn(pretrained = True)
model.eval()
# Setup up a simple transform that just converts to a tensor
transform = transforms.Compose([transforms.ToTensor()])
# Create variables for the labels for dog and cat
dog_label = 18
cat_label = 17

# Define the function to use the above as args
def get_boundbox_center(df_row_, label_=None, transform_=None, model_=None, debug=False):
  """
  For each url, detect an object (dog is label_ = 18; label_ = 17 for cat).
  If none is found, return [None, None, None].
  Else, return [True, x, y], where x and y are the center coordinates of the bounding
  box around the largest detected bounding box pertaining to the object category.

  Arguments:
  df_row_ (pd.Series): the row of the data frame (from df.DataFrame.apply)
    from which an url will be taken pointing to an image (at index 'FEATURE')
  label_ (int): The label consistent with the object category output by model
  transform_ (torch.transform): Transformation for image prior to model prediction
  model_ (torch): Pretrained object-detection model
  debug (bool): If True, a global variable called `debug_df` will contain the
    shape of each tensor after transformation of each image prior to prediction.

  Value
  None. Nothing is returned because this was causing a connection timeout.
  Instead a `center_points` (data frame) is created in the global env, built
   row by row with the results for each image, similar to debug_df.
  """
  if 'center_points' not in globals().keys():
    global center_points
    center_points = pd.DataFrame(columns = ['id', 'label', 'has_label', 'x', 'y'])

  # Get id of image
  id_ = df_row_['id']
  # Get URL
  url_ = df_row_['FEATURE']
  img_file = requests.get(url_)
  img = Image.open(BytesIO(img_file.content))

  # Ensure RGB color format (will ensure tensor shape is [3 channels, height, width])
  if img.mode != 'RGB':
    #print('Image Mode:', image.mode)
    img = img.convert('RGB')

  # Convert to tensor using supplied transform
  img_tensor = transform_(img) # transform_ arg to fn call

  # Update shapes with the image id and the shape of the tensor
  if debug:
    if 'debug_df' not in globals().keys():
      global debug_df
      debug_df = pd.DataFrame(columns = ['id', 'c', 'h', 'w'])
    new_shapes_row = pd.DataFrame([[df_row_['id']] + list(img_tensor.shape)],
                                  columns=debug_df.columns)
    debug_df = pd.concat([debug_df, new_shapes_row], axis = 0)

  # object detection
  with torch.no_grad():
      predictions = model_([img_tensor]) # model_ arg to fn call

  # Extract bounding box coordinates and labels
  boxes = predictions[0]['boxes'].cpu().numpy()
  labels = predictions[0]['labels'].cpu().numpy()

  # Find the largest bounding box (if any)
  centers = [] # center of each bounding box for this image
  #labels = [] # T/F if the bounding box for this image contains a dog
  areas = [] # area of each bounding box in this image.
  for box, label in zip(boxes, labels):
    if label == label_:
      # Get the center of the bounding box.
      x_min, y_min, x_max, y_max = map(int, box)
      x = x_min + round((x_max - x_min) / 2) # midpont
      y = y_min + round((y_max - y_min) / 2) # midpoint
      centers.append([x, y])
      # Get the bounding box area
      area = (x_max - x_min) * (y_max - y_min)
      areas.append(area)

  # If any label was dog, centers (and areas) are not empty
  if len(centers) > 0:
    # Find the largest bounding box labelled dog
    center = centers[np.nanargmax(areas)]
    x_ = int(center[0])
    y_ = int(center[1])
    ctr_row = pd.DataFrame([[id_, label_, True, x_, y_]], columns = center_points.columns)
    center_points = pd.concat([center_points, ctr_row], axis = 0)
  else:
    ctr_row = pd.DataFrame([[id_, label_, False, None, None]], columns = center_points.columns)
    center_points = pd.concat([center_points, ctr_row], axis = 0)



      #cropped_img_tensor = img_tensor[:, y_min:y_max, x_min:x_max]

      # Convert the cropped tensor back to a PIL image to plot
      #cropped_image_pil = T.ToPILImage()(cropped_img_tensor)
      # save image after cropping
      # cropped_image_pil.save('cropped_dog.jpg')

In [ ]:
# Read in the dogs and cats with url leading to found images.

# This is the data that only has rows for images that had links that worked
fdog = pd.read_csv('data/tbl/dog_good-url.csv', index_col = None)
fcat = pd.read_csv('data/tbl/cat_good-url.csv', index_col = None)

(4439, 24)

In [ ]:
# Test fn first
if 'debug_df' in globals().keys():
  del debug_df
if 'center_points' in globals().keys():
  del center_points
fdog.head(2).apply(get_boundbox_center,
                   args = (dog_label, transform, model, True),
                   axis = 1)
print(debug_df)
print(center_points)
if 'debug_df' in globals().keys():
  del debug_df
if 'center_points' in globals().keys():
  del center_points
fcat.head(2).apply(get_boundbox_center,
                   args = (cat_label, transform, model, True),
                   axis = 1)
print(debug_df)
print(center_points)

       id  c    h    w
0  2zfdiu  3   78  140
0  2zfi9w  3  140  140
       id label has_label   x   y
0  2zfdiu    18      True  72  40
0  2zfi9w    18      True  57  95
       id  c    h    w
0  3007wa  3  140  140
0  33fjm4  3  140  140
       id label has_label   x   y
0  3007wa    17      True  65  68
0  33fjm4    17      True  70  82


#### Run object detection

In [ ]:
# Dog
if 'debug_df' in globals().keys():
  del debug_df
if 'center_points' in globals().keys():
  del center_points
# This way it will go until connection error and save progress either way
# If it errors, it will save and then overwrite with the same thing, fine
try:
  fdog.apply(get_boundbox_center,
             args = (dog_label, transform, model, True),
             axis = 1)
except:
  if 'center_points' in globals().keys():
    center_points.to_csv('data/tbl/dog-centers_good-url.csv', index = False)
  if 'debug_df' in globals().keys():
    debug_df.to_csv('data/tbl/dog_center-points_debug.csv', index = False)
finally:
  if 'center_points' in globals().keys():
    center_points.to_csv('data/tbl/dog-centers_good-url.csv', index = False)
  if 'debug_df' in globals().keys():
    debug_df.to_csv('data/tbl/dog_center-points_debug.csv', index = False)
if 'debug_df' in globals().keys():
  print(debug_df.c.value_counts())

3    2886
Name: c, dtype: int64


In [ ]:
# Cat
if 'debug_df' in globals().keys():
  del debug_df
if 'center_points' in globals().keys():
  del center_points
# Try it
try:
  fcat.apply(get_boundbox_center,
             args = (cat_label, transform, model, True),
             axis = 1)
except:
  if 'center_points' in globals().keys():
    center_points.to_csv('data/tbl/cat-centers_good-url.csv', index = False)
  if 'debug_df' in globals().keys():
    debug_df.to_csv('data/tbl/cat_center-points_debug.csv', index = False)
finally:
  if 'center_points' in globals().keys():
    center_points.to_csv('data/tbl/cat-centers_good-url.csv', index = False)
  if 'debug_df' in globals().keys():
    debug_df.to_csv('data/tbl/cat_center-points_debug.csv', index = False)
if 'debug_df' in globals().keys():
  print(debug_df.c.value_counts())

3    4439
Name: c, dtype: int64


#### Save final dataset with usable image URLs

1. If the image exists (`image_found`)
2. If the image contains a dog or cat (from object detector)

In [ ]:
# Read in the data of centers for each image that had a good URL
dog_centers = pd.read_csv("data/tbl/dog-centers_good-url.csv", index_col = None)
cat_centers = pd.read_csv("data/tbl/cat-centers_good-url.csv", index_col = None)
# Change all id columns to int
for df_ in [dog_centers, cat_centers, fdog, fcat]:
  df_['id'] = df_['id'].astype('string')
# Prior to inner join, drop centers where dog/cat is not identified
dc = dog_centers[dog_centers['has_label']]
cc = cat_centers[cat_centers['has_label']]
# Inner join with the data
dog_final = fdog.merge(dc, left_on = 'id', right_on = 'id', how = 'inner').reset_index(drop = True)
dog_final.to_csv("data/tbl/dog-final-data_upvote-ratio-model.csv")
cat_final = fcat.merge(cc, left_on = 'id', right_on = 'id', how = 'inner').reset_index(drop = True)
cat_final.to_csv("data/tbl/cat-final-data_upvote-ratio-model.csv")